In [0]:
from pyspark.sql import SparkSession 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import sha2, concat_ws, upper, col, expr, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, LongType, StringType, DateType, IntegerType

In [0]:
spark = SparkSession.builder.appName('FactSales2Dimensionality').getOrCreate()

In [0]:
df_raw = spark.read.format("delta").load("/delta/fact_sales_final")

In [0]:
display(df_raw)

ProductCategory,ProductName,Brand,StoreRegion,StoreName,StoreType,SalesRep,Department,EmployeeRole,UnitsSold,UnitPrice,Discount,SaleDate
Furniture,T-shirt,BrandB,East,StoreX,Franchise,Martha Long,Electronics,Cashier,12.0,279.35,5.0,2022-12-14
Clothing,Tablet,BrandC,East,StoreZ,Franchise,Martha Long,Home,Sales Associate,33.67,272.49,0.0,2023-02-24
Clothing,Tablet,BrandA,South,StoreX,Retail,Emily Vazquez,Apparel,Cashier,33.67,484.75,15.0,2025-03-24
Electronics,Smartphone,BrandB,West,StoreY,Outlet,Charles Fields,Apparel,Cashier,26.0,205.74,10.0,2023-09-30
Furniture,T-shirt,BrandC,East,StoreZ,Outlet,Wendy Castillo,Home,Manager,46.0,20.25,5.0,2022-10-14
Furniture,T-shirt,BrandC,South,StoreY,Retail,Wendy Castillo,Home,Manager,22.5,361.06,10.0,2024-02-23
Clothing,T-shirt,BrandC,South,StoreY,Outlet,John Harris,Home,Cashier,37.0,492.65,5.0,2024-05-06
Electronics,Smartphone,BrandC,South,StoreX,Outlet,Charles Fields,Home,Sales Associate,37.0,293.87,15.0,2023-04-04
Clothing,Jeans,BrandA,South,StoreY,Retail,Wendy Castillo,Electronics,Manager,23.0,189.47,15.0,2022-12-26
Furniture,T-shirt,BrandB,East,StoreZ,Franchise,Charles Fields,Apparel,Manager,25.0,359.08,10.0,2022-10-28


In [0]:
df_cleaned = spark.read.format("delta").load("/delta/df_fact_cleaned")

In [0]:
display(df_cleaned)

DIM_ProductId,DIM_StoreId,DIM_EmployeeId,UnitsSold,UnitPrice,Discount,SaleDate,NetRevenue
0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,3c7aa2361bb113e0892437ecba18d0fca00119aefb95552a6b18bb7eee14759a,7e6b7470e0c5045a1797405ca77f04a0ba3c1e2e2e88bf125d9a33d10b597eac,12.0,279.35,5.0,2022-12-14,-13408.800000000001
c37d4ae373d30210c6e3b8760d2e14d3f1ce92979c432aa3ccef3493b5ef97b8,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,0f0a2f01281855e40cd8674f9192e6736b34aa8fc5f0478d71361f8a8412ba56,33.67,272.49,0.0,2023-02-24,9174.7383
a8406b374712399cebdf16a3e8a8c56cf72bf72db75cb2ad0dc3a1ff71e61285,d99aca1849c5560475ec973a6220e019f1b32f02cf85be1fdad144ec31466970,fcabe27a77b7b0062b0c99723dbfa1e531629cccb118cb71f9c41d22a9aba099,33.67,484.75,15.0,2025-03-24,-228501.45500000002
6a421110b266104bf1cf67be336c9ca4cfff1f897b231e0631948bb3f034677e,84b0247bf1c9e21f1a7befa453f14c708b5be10d1f352da6b89183f8d691ecc2,c93c0a4829fd929a76a46896a25c78a5e14ba88c21b0e526dbc45c4e5378ea90,26.0,205.74,10.0,2023-09-30,-48143.159999999996
8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,4bdb0c80a77b7ad25194de65cb257a855aef4714db3c3562712c670228caa9da,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab,46.0,20.25,5.0,2022-10-14,-3726.0
8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab,22.5,361.06,10.0,2024-02-23,-73114.65000000001
86ebe6b6839b1f0b802dc9acb8b3ac28ed84edd9496e774602b4bb858ac1ee21,411dea8e689cd912cd11682ee1f8b3ddbe7a4b413e89be652757c39f04f2ec60,fdedff2cb72a6a9be5669b3ceb5a76b5954802411dda3d1e83ddb183d5a86548,37.0,492.65,5.0,2024-05-06,-72912.2
fc08e15f7ca6c6df9df7da9e4815aaf89a29e1e19459ee7d969019e8cbb37123,76868e571d2e34422cf3c39e724b18da8fe589d29e3bd589ee5390c548aaabd5,27d0986b4172fcb4be66444087e6bced12fe736c8e1a53d6f5144aea5778742c,37.0,293.87,15.0,2023-04-04,-152224.66
807d270019a32fa7c40a2b8931312f136363e6bf9793576c7ab67fd901472887,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,5b35a50e37f2fb896eef5da16148195eff3e208575ad22d8a6edfbff8511201c,23.0,189.47,15.0,2022-12-26,-61009.340000000004
0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,11e3df9450f3951d15425406cfd773e170e7e53297b071b8dfcfa87a688599c3,25.0,359.08,10.0,2022-10-28,-80793.0


Selecting only columns that are common in both

In [0]:
fact_columns = ["UnitsSold", "UnitPrice", "Discount", "SaleDate"]
df_raw_selected = df_raw.select(fact_columns)
df_cleaned_selected = df_cleaned.select(fact_columns)

In [0]:
df_raw_selected = df_raw_selected.withColumn("row_id", monotonically_increasing_id())
df_cleaned_selected = df_cleaned_selected.withColumn("row_id", monotonically_increasing_id())

In [0]:
df_compare = df_raw_selected.alias("raw").join(
    df_cleaned_selected.alias("cleaned"),
    on="row_id",
    how="inner"
)

df_mismatch = df_compare.filter(
    (col("raw.UnitsSold") != col("cleaned.UnitsSold")) |
    (col("raw.UnitPrice") != col("cleaned.UnitPrice")) |
    (col("raw.Discount") != col("cleaned.Discount")) |
    (col("raw.SaleDate") != col("cleaned.SaleDate"))
)

df_mismatch.show(truncate=False)

+------+---------+---------+--------+--------+---------+---------+--------+--------+
|row_id|UnitsSold|UnitPrice|Discount|SaleDate|UnitsSold|UnitPrice|Discount|SaleDate|
+------+---------+---------+--------+--------+---------+---------+--------+--------+
+------+---------+---------+--------+--------+---------+---------+--------+--------+



Seems there are no mismatch, lets print it out for usability.

In [0]:
print("Total Rows Compared:", df_compare.count())
print("Mismatched Rows:", df_mismatch.count())

Total Rows Compared: 30
Mismatched Rows: 0
